# phitigra: a simple graph editor

[SageMath](https://www.sagemath.org/) has a large set of functions for [graph theory](https://doc.sagemath.org/html/en/reference/graphs/index.html). Defining graphs by hand can however be complicated as vertices and edges are added with the command line.
This package is an editor that allows to define or change graphs using the mouse. It has the form of a [Jupyter](https://jupyter.org/) widget.

## Getting started


In [ ]:
from phitigra import GraphEditor

The editor widget is a `GraphEditor` object. By default the canvas is empty; you can add vertices and edges by clicking on *add vertex or edge* and clicking on the canvas.

In [ ]:
editor = GraphEditor()
editor.show()

It is also possible to plot (and later edit) an already existing graph. Note that the two instances of the editor are completely independent.

In [ ]:
G = graphs.PetersenGraph()
editor2 = GraphEditor(G)
editor2.show()

Now you can move vertices, change their color, etc. The graph drawn can be accessed with `.graph`. It is the same object as the graph given when creating the widget.

In [ ]:
editor2.graph

In [ ]:
editor2.graph is G

A copy of the drawn graph can be obtained as follows:

In [ ]:
H = editor2.get_graph()
H == G and not H is G

### Application 1: testing a conjecture

In [ ]:
def conjecture(G):
    return not G.is_vertex_transitive() or G.is_hamiltonian()

Let us [conjecture](https://en.wikipedia.org/wiki/Lov%C3%A1sz_conjecture#Hamiltonian_cycle) that every vertex transitive graph is hamiltonian. Then `conjecture(G)` should return `True` for every graph `G`. We can to test in on various small graphs drawn in the widget. If the graph in the above widget is still the Petersen graph, the following should return `False`, disproving the conjecture.

In [ ]:
conjecture(editor2.get_graph())

### Application 2: producing pictures for your papers

The drawing of the graph in the editor can be exported to a latex (tikz) picture to be included in a paper. The latex code can be obtained as follows: 

In [ ]:
latex(editor2.graph)

Note that only the positions of the vertices will be kept. See [this page](https://doc.sagemath.org/html/en/tutorial/latex.html#an-example-combinatorial-graphs-with-tkz-graph) for more details about exporting graphs to latex. The resulting pdf image can be seen as follows.

In [ ]:
view(editor2.graph)

Note that the above requires ``pdflatex``. It  will fail if you run this demo on binder.

## Widget settings

Several parameters of the widget can be changed:
  * the width and height of the drawing canvas;
  * the default radius and color for vertices;
  * the default color for edges.

In [ ]:
editor3 = GraphEditor(graphs.PetersenGraph(), width=300, height=300, default_radius=12, default_vertex_color='orange', default_edge_color='#666')
editor3.show()

## Changing the drawing

Changes to the drawing can be done with the mouse of course, but also by calling appropriate functions.

In [ ]:
K = graphs.RandomGNP(10, 0.5)
editor4 = GraphEditor(K)
editor4.show()

In [ ]:
editor4.set_vertex_color(1, 'white') # recolor vertex 1
editor4.set_vertex_radius(6, 25)     # make vertex 6 bigger
editor4.refresh()                    # needed to update the canvas

### Application 4: automatically setting colors

The colors of the vertices and edges can be defined by a function.

In [ ]:
for v in editor4.graph:
    # Radius depends on degree:
    editor4.set_vertex_radius(v, editor4.graph.degree(v)*3+5)
    # Color depends on parity
    if is_odd(v):
        editor4.set_vertex_color(v, '#4040dd')
    else:
        editor4.set_vertex_color(v, 'green')

for e in editor4.graph.edge_iterator():
    u, v, _ = e
    if is_odd(u + v):
        editor4.set_edge_color((u,v), 'orange')
    else:
        editor4.set_edge_color((u,v), 'purple')
editor4.refresh()

Now the size of the vertices shapes in the widget above depend on the degree of the vertices and the color of the vertices and edges depend on the parity of the vertices names.

### Application 5: making animations

The following code will draw a random walk of length 30 in the widget above, with colors of the vertices gradually becoming lighter. Run the cell and scroll up fast to see the result!

In [ ]:
from random import choice
from time import sleep

def col(i, n):
    # Return a color depending on i
    rgbv = int(i * 255 / n)
    return '#%02x%02x%02x' % (100, rgbv , 255 - rgbv)

# Reset the colors and sizes on the widget above
for v in editor4.graph:
    editor4.set_vertex_color(v, 'white')
    editor4.set_vertex_radius(v, 15)
for e in editor4.graph.edge_iterator():
    editor4.set_edge_color(e, 'black')

n=30    
v = choice(editor4.graph.vertices())
editor4.set_vertex_color(v, col(0, n))
editor4.refresh()
sleep(2)

for i in range(1, n):
        sleep(float(0.5)) # wait
        u = choice(editor4.graph.neighbors(v))
        color = col(i, n)
        editor4.set_vertex_color(u, color)
        editor4.set_edge_color((v, u), color)
        v = u
        editor4.refresh()
    

## Calling a custom function by mouse clicks

There is a *Next* button on the widget. It can be set to call a custom function that changes the drawn graph.

Below we define a function that takes as input a graph editor widget and performs the following on its graph:
  
  * when first called, it choses a random vertex, colors it and stores it as a private variable of the widget;
  * on the subsequent calls, it choses a random neighbor of the last previously chosen vertex that has not been considered yet, if any, colors it and stores it as well in the widget object.
  
By registering this function as the callback for the *Next* button, we are able to see the different steps of the construction of a maximal path by clicking repeteadly on *Next*.


In [ ]:
from random import choice

def greedy_path(widget):
    # Greedily construct path starting at a random vertex
    if not hasattr(widget, '_path'):
        first = choice(widget.graph.vertices())   # The path starts at a random vertex
        widget._path = [first]                    # We use the widget to store the state of the algorithm
        widget.set_vertex_color(first, 'orange')
        widget.refresh()
        return
    last = widget._path[-1]
    
    free_neighbors = [v for v in widget.graph.neighbor_iterator(last) if not v in widget._path]
    if not free_neighbors:
        widget.output_text('The path cannot be extended!')
    else:
        nextv = choice(free_neighbors)  
        widget._path.append(nextv)
        widget.set_vertex_color(nextv, 'red')
        widget.set_edge_color((last, nextv), 'red')
        widget.refresh()    

In [ ]:
R = graphs.RandomGNP(10, 0.5)
editor5 = GraphEditor(R, default_vertex_color = 'white')
editor5.show()

In [ ]:
editor5.set_next_callback(greedy_path) # Define the callback for the 'Next' button

Now clicks on *Next* update the graph !

### Application 6: presenting algorithms

The callback for the *Next* button can be used to run an algorithm step-by-step on an example graph during a lecture. For instance to display the computation of a shortest path, a spanning tree, a BFS search, etc.

### Application 7: testing algorithms

It can also be used to try a function on small instances or to debug.

In [ ]:
editor6 = GraphEditor(graphs.PetersenGraph(), default_vertex_color = 'white')
editor6.show()

In [ ]:
def mdscolor(widget):
    "Color the vertices with minimal dominating sets"
    if not hasattr(widget, '_mds'):
        widget._mds = widget.graph.minimal_dominating_sets() # an iterator over minimal dominating sets
        widget.output_text('Showing the minimal dominating sets...')
    
    try:
        D = next(widget._mds)
    except StopIteration:
        widget.output_text('Done! Back to the initial state.')
        for v in widget.graph:
            widget.set_vertex_color(v, 'white')
        widget.refresh()        
        del widget._mds
        return
    
    for v in widget.graph:
        widget.set_vertex_color(v, 'white')
    for v in D:
        widget.set_vertex_color(v, 'green')
    widget.refresh()

# Register the function as a callback for the `Next` button:
editor6.set_next_callback(mdscolor)

Now you can click on *Next* on the widget above. After each click, a new minimal dominating set of the graph will be shown with green vertices!